In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { margin-left: 0 !important; width:100% !important; }</style>"))

In [ ]:
import math
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
import json
from types import SimpleNamespace
import cmath
import glob
import itertools
import tensorboard
import time
import traceback

In [ ]:
tf.keras.mixed_precision.set_global_policy('float64')

In [ ]:
# example0:
# {
#   "EntryType":"EquivalentLoudnessMeasurement",
#   "Calibration":{
#     "HeadphoneFrequencyResponseHash":"eab807a5050b0087109ac1fd6fecbd09197a3dcd",
#     "FullScaleSineDBSPL":100
#   },
#   "Run":{
#     "ID":"1599737802659_2857050667"
#   },
#   "Evaluation":{
#     "ID":"1599738237070_2365796582",
#     "Frequency":697.9866246810275,
#     "Probe":{
#       "Type":"Noise",
#       "Params":{
#         "Level":-10,
#         "LowerLimit":966.84025,
#         "Onset":{
#           "Delay":0,
#           "Duration":0.1
#         },
#         "UpperLimit":1033.15975
#       }
#     },
#     "Combined":{
#       "Type":"Superposition",
#       "Params":[
#         {
#           "Params":{
#             "Level":-40,
#             "LowerLimit":966.84025,
#             "Onset":{
#               "Delay":0.5,
#               "Duration":0.1
#             },
#             "UpperLimit":1033.15975
#           },
#           "Type":"Noise"
#         },{
#           "Params":{
#             "Level":-20,
#             "LowerLimit":672.9766301106662,
#             "Onset":{
#               "Delay":0,
#               "Duration":0.1
#             },
#             "UpperLimit":722.9966192513888
#           },
#           "Type":"Noise"
#         }
#       ]
#     }
#   },
#   "Results":{
#     "ProbeGainForEquivalentLoudness":0.003107877399956887,
#     "ProbeDBSPLForEquivalentLoudness":39.84927756697657
#   }
# }
def experiment_result_to_example(exp, car_fs_sine_db_spl=100, window_size=2048, sample_rate=48000, noise_floor_db_spl=30):
    bin_width = sample_rate / window_size
    if exp.EntryType != 'EquivalentLoudnessMeasurement':
        return None
    assert exp.Evaluation.Probe.Type == 'Noise'
    assert exp.Evaluation.Combined.Type == 'Superposition'
    for part in exp.Evaluation.Combined.Params:
        assert part.Type == 'Noise'
        # If this part isn't the probe.
        if (part.Params.LowerLimit != exp.Evaluation.Probe.Params.LowerLimit or
            part.Params.UpperLimit != exp.Evaluation.Probe.Params.UpperLimit):
            # We want to ignore evaluations where probe and masker overlap.
            if ((part.Params.LowerLimit + bin_width * 2 >= exp.Evaluation.Probe.Params.LowerLimit and
                 part.Params.LowerLimit - bin_width * 2 <= exp.Evaluation.Probe.Params.UpperLimit) or
                (part.Params.UpperLimit + bin_width * 2 >= exp.Evaluation.Probe.Params.LowerLimit and
                 part.Params.UpperLimit - bin_width * 2 <= exp.Evaluation.Probe.Params.UpperLimit) or
                (part.Params.LowerLimit - bin_width * 2 <= exp.Evaluation.Probe.Params.LowerLimit and
                 part.Params.UpperLimit + bin_width * 2 >= exp.Evaluation.Probe.Params.UpperLimit)):
                return None
    assert exp.Results.ProbeGainForEquivalentLoudness > 0
    assert exp.Results.ProbeDBSPLForEquivalentLoudness != None
    assert exp.Calibration.FullScaleSineDBSPL > 0

    probe_fs = [exp.Evaluation.Probe.Params.LowerLimit, exp.Evaluation.Probe.Params.UpperLimit]
    combined_sigs = []
    for part in exp.Evaluation.Combined.Params:
        part_db_spl = exp.Calibration.FullScaleSineDBSPL + part.Params.Level
        part_db_fs = part_db_spl - car_fs_sine_db_spl
        combined_sigs.append([part_db_fs, part.Params.LowerLimit, part.Params.UpperLimit])
    return synthesize_example(probe_fs, exp.Results.ProbeDBSPLForEquivalentLoudness, combined_sigs, noise_floor_db_spl - car_fs_sine_db_spl, window_size=window_size, sample_rate=sample_rate)

def make_noise(total_power_db_fs, lower_limit, upper_limit, window_size=2048, sample_rate=48000):
    np.random.seed(0)
    # Power of full scale sine is variance, which is 0.5
    fs_offset = -10 * np.log10(0.5)
    bin_width = sample_rate / window_size
    first_bin = int(np.floor(lower_limit/bin_width))
    last_bin = int(np.ceil(upper_limit/bin_width))
    noise_coeffs = np.zeros(shape=[window_size//2], dtype=np.complex128)
    noise_coeffs[first_bin:last_bin+1] = (complex(0, 1) * np.random.normal(size=[last_bin-first_bin+1]) + np.random.normal(size=[last_bin-first_bin+1]))
    noise_coeffs = np.concatenate([noise_coeffs, [0], np.flip(np.conj(noise_coeffs[1:]))])
    noise_signal = np.real(np.fft.ifft(noise_coeffs))
    noise_power_db_fs = 10 * np.log10(np.var(noise_signal)) + fs_offset
    scale = 10 ** ((total_power_db_fs - noise_power_db_fs) / 20)
    noise_signal *= scale
    new_power = 10 * np.log10(np.var(noise_signal)) + fs_offset
    if np.abs(new_power - total_power_db_fs) > 1e-6:
        raise ValueError(f'wrong power, got {new_power} wanted {total_power_db_fs}')
    return noise_signal

# probe_fs are [lowest_freq_hz, highest_freq_hz]
# combined_sigs are [[intensity_db_fs, lowest_freq_hz, highest_freq_hz]]
def synthesize_example(probe_fs, equiv_intensity_db_fs, combined_sigs, noise_floor_db_fs, window_size=2048, sample_rate=48000):
    bin_width = sample_rate / window_size

    signal = make_noise(noise_floor_db_fs, 20, 14000, window_size=window_size)
    
    for part in combined_sigs:
        signal += make_noise(part[0], part[1], part[2], window_size=window_size)
    
    relevant_bins = np.zeros([int(window_size/2)], dtype=np.float64)
    first_relevant_bin = int(np.floor(probe_fs[0]/bin_width))
    last_relevant_bin = int(np.ceil(probe_fs[1]/bin_width))
    relevant_bins[first_relevant_bin:last_relevant_bin+1] = 1

    true_loudness = np.array([equiv_intensity_db_fs], dtype=np.float64)
    
    res = tf.concat([signal, true_loudness, relevant_bins], axis=0)
    return res[:,None]

def prepare_examples(examples):
    filtered_iter = filter(lambda e: e != None, examples)
    ds = tf.data.Dataset.from_tensor_slices(list(filtered_iter))
    input_true_ds = ds.map(lambda ex: (ex[:window_size], ex[window_size:]))
    return input_true_ds

def load_examples(glb, car_fs_sine_db_spl=100, window_size=2048, sample_rate=48000):
    lines_iter = itertools.chain(*map(lambda f: open(f).readlines(), glob.glob(glb)))
    json_iter = map(lambda l: json.loads(l, object_hook=lambda o: SimpleNamespace(**o)), lines_iter)
    only_eval_iter = filter(lambda exp: exp.EntryType == 'EquivalentLoudnessMeasurement', json_iter)
    sorted_iter = sorted(only_eval_iter, key=lambda exp: exp.Evaluation.ID)
    examples_iter = map(lambda l: experiment_result_to_example(l, car_fs_sine_db_spl=car_fs_sine_db_spl, window_size=window_size, sample_rate=sample_rate), sorted_iter)
    return prepare_examples(examples_iter)
            
def plot_pz(f):
    _, ax = plt.subplots(figsize=(4,4))
    ax.add_patch(patches.Circle((0,0),
                              radius=1,
                              fill=False,
                              color='black',
                              ls='solid',
                              alpha=0.1))
    ax.axvline(0, color='0.7')
    ax.axhline(0, color='0.7')
    ax.set_xlim((-1.1,1.1))
    ax.set_ylim((-1.1,1.1))

    ax.plot(tf.reshape(tf.math.real(f.poles), [-1,1]), tf.reshape(tf.math.imag(f.poles), [-1,1]),
            'x', markersize=9, alpha=0.5)
    ax.plot(tf.reshape(tf.math.real(f.zeros), [-1,1]), tf.reshape(tf.math.imag(f.zeros), [-1,1]),
            'o', color='none', markeredgecolor='red',
            markersize=9, alpha=0.5)
    plt.tight_layout()
    plt.show()

def plot_many_z(many_z, sample_rate=48000, figsize=(12,4)):
    xaxis = np.tile(np.linspace(0,
                                (many_z.shape[1] - 1) * sample_rate *
                                0.5 / many_z.shape[1],
                                many_z.shape[1]), [many_z.shape[0], 1])
    _, ax = plt.subplots(figsize=figsize)    
    ax.set_xscale('log')
    ax.set_xlim((10,20000))
    ax.set_ylim((-20, 70))
    x = np.transpose(xaxis[:,:xaxis.shape[1]//2])
    y = np.transpose(20 * np.log10(1e-20+np.abs(many_z[:,:many_z.shape[1]//2])))
    ax.plot(x, y)
    plt.tight_layout()
    plt.show()
        
def plot_snr(snr, channels=[None, None], bins=[None, None]):
    max_snr = np.nanmax(snr)
    min_snr = np.nanmin(snr)
    scaled_snr = ((snr - min_snr) / (max_snr - min_snr))[:,:,None]
    if channels[1] != None:
        scaled_snr = scaled_snr[:channels[1]]
    if channels[0] != None:
        scaled_snr = scaled_snr[channels[0]]
    if bins[1] != None:
        scaled_snr= scaled_snr[:,:bins[1]]
    if bins[0] != None:
        scaled_snr = scaled_snr[:,bins[0]:]
    scaled_snr = np.repeat(scaled_snr, 4, axis=0)
    pixels = np.concatenate([scaled_snr, scaled_snr/2, scaled_snr/2], axis=2)
    plt.figure(figsize = (10,5))
    plt.imshow(pixels, interpolation='bicubic')
    plt.tight_layout()
    plt.show()
    
def plot_pred(model, ds):
    model = tf.function(model)
    truth = []
    pred = []
    worst_example = None
    worst_error = None
    worst_index = None
    example_index = 0
    for exIdx, ex in enumerate(ds):
        x, y_true = ex
        y_true = tf.reshape(y_true, [y_true.shape[0], y_true.shape[1]])
        for val in y_true[:,0]:
            truth.append(np.abs(val))
        y_pred = model(x)
        pred_min = np.min(y_pred, axis=1)[:,None]
        irrelevant_reduced = np.where(np.abs(y_true[:,1:]) != 0, y_pred, pred_min-1)
        for idx, val in enumerate(np.max(irrelevant_reduced, axis=1)):
            pred.append(val)
            err = (truth[idx] - val) ** 2 * truth[idx]            
            if worst_error is None or err > worst_error:
                worst_error = err
                worst_example = x[idx]
                worst_index = example_index
            example_index += 1
    plt.figure(figsize=(10,5))
    steps = np.arange(len(truth))
    plt.plot(steps, truth, label='truth')
    plt.plot(steps, pred, label='pred')
    plt.legend()
    plt.tight_layout()
    plt.show()
    return (worst_index, worst_error, worst_example)

# (a - x)*(b - x)*... => k1 + k2*x + k3*x^2 + ... 
# This function takes [a,b,...] and produces k1,k2,k3,...
@tf.function(experimental_relax_shapes=True)
def coeffs(constants, length=None):
    if length is None:
        length = constants.shape[0]
    res = tf.TensorArray(size=length+1, dtype=tf.complex128)
    for num in range(length+1):
        if num == 0:
            s = tf.constant(1.0, dtype=tf.complex128)
        else:
            s = tf.constant(0.0, dtype=tf.complex128)
            for parts in itertools.combinations(np.arange(length), num):
                prod = tf.constant(1.0, dtype=tf.complex128)
                for part in parts:
                    prod *= -constants[part]
                s += prod
        res = res.write(num, s)
    return res.stack()

class RNNLayer(tf.keras.layers.Layer):
    def __init__(self, cell, **kwargs):
        super().__init__(**kwargs)
        self.cell = cell
    @tf.function(experimental_relax_shapes=True)
    def call(self, inputs, **kwargs):
        state = tuple(tf.zeros(shape=[inputs.shape[0], size], dtype=self.dtype) for size in self.cell.state_size)
        def cond(step, output, state):
            return step < inputs.shape[1]
        def body(step, output, state):
            step_input = inputs[:,step]
            step_output, state = self.cell.call(step_input, state)
            output = output.write(step, step_output)
            return step+1, output, state
        _, output, self.state = tf.while_loop(
            shape_invariants=[
                tf.TensorShape(None), 
                tf.TensorShape(None), 
                tuple([s.shape for s in state])
            ],
            cond=cond, 
            body=body, 
            loop_vars=[
                0, 
                tf.TensorArray(size=inputs.shape[1], dtype=inputs.dtype), 
                state
            ],
        )
        output = output.stack()
        dims = np.arange(len(output.shape))
        dims[0] = 1
        dims[1] = 0
        output = tf.transpose(output, dims)
        return output

class PZCell:
    def __init__(self, gain, poles, zeros):
        self.gain = gain
        self.poles = poles
        self._n_poles = poles.shape[0] * 2
        self.zeros = zeros
        self._n_zeros = zeros.shape[0] * 2
        self.output_size = 1
        self.state_size = [2*self.poles.shape[0]+1, 2*self.poles.shape[0]+1]
    # [batch, 1], ([batch, steps_back], [batch, steps_back]) - 0 is at t-1 => [batch, 1], ([batch, steps_back], [batch, steps_back]) - 0 is at t
    @tf.function(experimental_relax_shapes=True)
    def call(self, input_at_t, states_at_t):
        print('pzc input, state', input_at_t, states_at_t)
        input_dtype = input_at_t.dtype
        states_dtype = states_at_t[0].dtype
        poles = tf.reshape(tf.concat([self.poles, tf.math.conj(self.poles)], axis=0), [self._n_poles])
        zeros = tf.reshape(tf.concat([self.zeros, tf.math.conj(self.zeros)], axis=0), [self._n_zeros])
        input_at_t = tf.cast(input_at_t, dtype=tf.complex128)
        x_memory = tf.cast(states_at_t[0], tf.complex128)
        y_memory = tf.cast(states_at_t[1], tf.complex128)
        x_memory = tf.concat([input_at_t, x_memory[:, :x_memory.shape[1]-1]], axis=1)
        pole_coeffs = coeffs(poles, self._n_poles)
        zero_coeffs = coeffs(zeros, self._n_zeros)
        zero_offset = tf.math.maximum(0, self._n_poles - self._n_zeros)
        output_at_t = tf.constant(0, shape=input_at_t.shape, dtype=tf.complex128)
        zero_components = x_memory[:,zero_offset:] * self.gain * zero_coeffs
        output_at_t += tf.math.reduce_sum(zero_components, axis=1)[:,None]
        pole_components = y_memory[:,:y_memory.shape[1]-1] * pole_coeffs[1:]
        output_at_t -= tf.math.reduce_sum(pole_components, axis=1)[:, None]
        output_at_t = tf.math.divide_no_nan(output_at_t, pole_coeffs[0])
        y_memory = tf.concat([output_at_t, y_memory[:, :y_memory.shape[1]-1]], axis=1)
        states_at_t_plus_1 = (tf.cast(tf.math.real(x_memory), states_dtype), tf.cast(tf.math.real(y_memory), states_dtype))
        output_at_t = tf.math.real(tf.reshape(output_at_t, [output_at_t.shape[0]]))
        output_at_t = tf.cast(output_at_t[:,None], input_dtype)
        #print('pzc output, state', output_at_t, states_at_t_plus_1)
        return (output_at_t, states_at_t_plus_1)
        
class CARCell:
    # See 'Human and Machine Hearing' (http://dicklyon.com/hmh/), 16.2-16.3,
    # and https://github.com/google/carfac/.
    def __init__(self, sample_rate=48000, erb_per_step=0.5, linear=False):
        self._erb_per_step = erb_per_step
        self._sample_rate = sample_rate
        self._linear = linear

        # Based on the assumtion that max small-signal gain at the passband peak
        # will be on the order of (0.5/min_zeta)^(1/erb_per_step), and we need
        # the start value of that in the same region or the loss function becomes
        # too uneven to optimize.
        def compute_zeta(zeta_at_default_erb_per_step, erb_per_step):
            default_erb_per_step = 0.5
            max_small_signal_gain = (0.5 /
                                     zeta_at_default_erb_per_step) ** (1 / default_erb_per_step)
            return 0.5 / (max_small_signal_gain ** erb_per_step)

        # Controls r (pole and zero abs value) which controls damping relative to
        # frequency.
        self._high_f_damping_compression = tf.Variable(
            name='high_f_damping_compression',
            dtype=tf.float64,
            initial_value=0.5)
        # Controls distance from pole to zero.
        self._zero_ratio = tf.Variable(
            name='zero_ratio',
            dtype=tf.float64,
            initial_value=(2 ** 0.5))
        # min/max zeta controls max damping.
        self._min_zeta = tf.Variable(
            name='min_zeta',
            dtype=tf.float64,
            initial_value=compute_zeta(0.1, self._erb_per_step))
        self._max_zeta = tf.Variable(
            name='max_zeta',
            dtype=tf.float64,
            initial_value=compute_zeta(0.35, self._erb_per_step))
        # Controls how we convert from Hz to Cams.
        # Defaults at numbers from B.C.J. Moore and B.R. Glasberg, "Suggested formulae for calculating auditory-filter bandwidths and excitation patterns".
        self._erb_constant_0 = tf.Variable(
            name='erb_constant_0(24.7)',
            dtype=tf.float64,
            initial_value=24.7)
        self._erb_constant_1 = tf.Variable(
            name='erb_constant_1(1.0)',
            dtype=tf.float64,
            initial_value=1)
        self._erb_constant_2 = tf.Variable(
            name='erb_constant_2(4.37)',
            dtype=tf.float64,
            initial_value=4.37)
        self._v_offset = tf.Variable(
            name='v_offset',
            dtype=tf.float64,
            initial_value=0.04)
        self._velocity_scale = tf.Variable(
            name='velocity_scale',
            dtype=tf.float64,
            initial_value=0.1)
        self._max_freq = 20000
        self._min_freq = 20
        f = self._max_freq
        self._n_poles = 1
        while f - self._erb_per_step * self.ERB(f) > self._min_freq:
            f = f - self._erb_per_step * self.ERB(f)
            self._n_poles += 1
            
        self.output_size = self._n_poles
        # [z1, z2]
        self.state_size = [self.output_size, self.output_size, self.output_size]

    # Equivalent rectangular bandwidth, the width of the theoretical rectangular cochlear place filter.
    def ERB(self, f):
        return self._erb_constant_0 * (self._erb_constant_1 + self._erb_constant_2 * f * 0.001)
    # [batch, 1], ([batch, prev_u], [batch, prev_v]) => [batch, channel], ([batch, u], [batch, v])
    @tf.function(experimental_relax_shapes=True)
    def call(self, input_at_t, states_at_t):
        #print('cc call input, state', input_at_t, states_at_t)
        input_dtype = input_at_t.dtype
        states_dtype = states_at_t[0].dtype
        
        input_at_t = tf.cast(input_at_t, dtype=tf.float64)

        pole_freqs_ta = tf.TensorArray(size=self._n_poles, dtype=tf.float64)
        f = self._max_freq
        for channel_number in range(self._n_poles):
            pole_freqs_ta = pole_freqs_ta.write(channel_number, f)
            f -= self._erb_per_step * self.ERB(f)
        pole_freqs = pole_freqs_ta.stack()
        #print('cc call pole_freqs', pole_freqs.shape)

        # From the matlab code:
        # zero_ratio comes in via h.  In book's circuit D, zero_ratio is 1/sqrt(a),
        # and that a is here 1 / (1+f) where h = f*c.
        # solve for f:  1/zero_ratio^2 = 1 / (1+f)
        # zero_ratio^2 = 1+f => f = zero_ratio^2 - 1
        f = tf.math.square(self._zero_ratio) - 1
        pole_thetas = pole_freqs * 2 * np.pi / self._sample_rate
        # The book assigns a0 and c0 thus to simplify the equations.
        a0 = tf.math.cos(pole_thetas)
        c0 = tf.math.sin(pole_thetas)
        #print('cc call a0 c0', a0.shape, c0.shape)

        # The ratio between each pole and max measurable frequency.
        x = pole_thetas / np.pi

        # From the matlab code:
        # When high_f_damping_compression is 0 this is just theta, when
        # high_f_damping_compression is 1 it approaches 0 as theta approaches pi.
        zr_coeffs = np.pi * (x - self._high_f_damping_compression * tf.math.pow(x, 3))

        # The book is not super easy to follow here, so I have mostly
        # implemented the same math as the matlab and c++ code.

        r1 = (1 - zr_coeffs * self._max_zeta)
        min_zetas = self._min_zeta + (0.25 * ((self.ERB(pole_freqs) / pole_freqs) - self._min_zeta))
        zr_coeffs *= (self._max_zeta - min_zetas)

        prev_u = tf.cast(states_at_t[0], tf.float64)
        prev_v = tf.cast(states_at_t[1], tf.float64)
        # This is called zA_memory in the matlab code, its a delay of prev_v.
        prev_vel_mem = tf.cast(states_at_t[2], tf.float64)

        # Velocity is the velocity of the pressure differential.
        velocity = prev_v - prev_vel_mem
        #print('car velocity', velocity.shape)
        # NLF is the nonlinearity doing the fast acting compression.
        nlf_out = 1.0 / (1.0 + tf.math.square(velocity * self._velocity_scale + self._v_offset))
        #print('car nlf_out', nlf_out.shape)
        # The undampening is controlled via the NLF.
        if self._linear:
            r = r1 + zr_coeffs * tf.ones(shape=velocity.shape, dtype=tf.float64)
        else:
            r = r1 + zr_coeffs * nlf_out
        #print('car r', r.shape)

        h = c0 * f
        g0 = (1 - 2 * r * a0 + tf.math.square(r)) / (1 - 2 * r * a0 + h * r * c0 + tf.math.square(r))
        
        # Following is based on matlab code and figures 16.1 and 17.1 from the book. In figure 17.1
        # I have used the names U and V in the same way as in figure 16.1.
        
        #print('cc call r, prev_u, prev_v', r.shape, prev_u.shape, prev_v.shape)
        # This doesn't include the input X
        partial_u = r * (a0 * prev_u - c0 * prev_v)
        # V doesn't include the input 
        v = r * (c0 * prev_u + a0 * prev_v)
        #print('cc call v, partial_u', v.shape, partial_u.shape)
        
        # This doesn't include the input X
        partial_y = h * v
        #print('cc call partial_y, g0', partial_y.shape, g0.shape)

        u_ta = tf.TensorArray(size=self._n_poles, dtype=tf.float64)
        output_at_t_ta = tf.TensorArray(size=self._n_poles, dtype=tf.float64)
        in_out = input_at_t[:,0]
        for ch in range(self._n_poles):
            #print('cc call partial_u[:,ch], in_out', partial_u[:,ch].shape, in_out.shape)
            # Add this input to partial_u to get actual u.
            u_ta = u_ta.write(ch, partial_u[:,ch] + in_out)
            # Add this input to partial_output to get actual output.
            in_out = tf.math.real(g0[:,ch]) * (in_out + partial_y[:, ch])
            # Save this actual output.
            output_at_t_ta = output_at_t_ta.write(ch, in_out)

        u = u_ta.stack()
        #print('cc call u', u.shape)
        u = tf.transpose(u, [1,0])
        #print('cc call u', u.shape)
        states_at_t_plus_1 = (tf.cast(u, states_dtype), tf.cast(v, states_dtype), tf.cast(prev_v, states_dtype))
        output_at_t = output_at_t_ta.stack()
        output_at_t = tf.cast(tf.transpose(output_at_t, [1,0]), input_dtype)
        #print('cc call output, states', output_at_t, states_at_t_plus_1)
        return (output_at_t, states_at_t_plus_1)
    
class FFTLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    @tf.function(experimental_relax_shapes=True)
    def call(self, inputs, **kwargs):
        return tf.math.abs(tf.signal.fft(tf.cast(inputs, tf.complex128)))
    
class SNRLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    # [...,fft_coeffs] => [...,channels,db_snr]
    @tf.function(experimental_relax_shapes=True)
    def call(self, inputs, **kwargs):
        gain = tf.math.abs(inputs)
        signal_power = tf.math.square(gain)
        noise_power = tf.math.reduce_sum(signal_power, axis=2)[:,:,None] - signal_power
        snr = tf.math.divide_no_nan(tf.math.square(signal_power), noise_power)
        res = tf.cast(10.0, self.dtype) * tf.cast(tf.math.log(snr + 1e-16), self.dtype) / tf.cast(tf.math.log(10.0), self.dtype)
        res = res[:,:,:res.shape[2]//2]
        return res

class LoudnessPredictorLoss(tf.keras.losses.Loss):
    @tf.function(experimental_relax_shapes=True)
    def call(self, y_true, y_pred):
        true_partial_loudness = y_true[:,0,0]
        relevant_bins = tf.cast(y_true[:,1:,0], tf.bool)
        min_psnr = tf.math.reduce_min(y_pred, axis=1)[:,None]
        reduced_irrelevant_bins = tf.where(relevant_bins, y_pred, min_psnr - 1)
        psnr_across_relevant_bins = tf.math.reduce_max(reduced_irrelevant_bins, axis=1)
        res = tf.cast(tf.keras.losses.MSE(true_partial_loudness, psnr_across_relevant_bins), true_partial_loudness.dtype) * true_partial_loudness
        return res

class LoudnessPredictorLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._loudness_offset = self.add_weight(
            name='loudness_offset',
            initializer=tf.keras.initializers.Constant(1),
            trainable=self.trainable,
        )
        self._loudness_scale = self.add_weight(
            name='loudness_scale',
            initializer=tf.keras.initializers.Constant(1),
            trainable=self.trainable,
        )
    # [...,fft_bin] => [...,equivalent_db_spl]
    @tf.function(experimental_relax_shapes=True)
    def call(self, inputs, **kwargs):
        psnr_per_bin = tf.math.reduce_max(inputs, axis=1)
        res = self._loudness_offset * -110 + self._loudness_scale *3 * psnr_per_bin
        return res

class EarLayer(RNNLayer):
    def __init__(self, **kwargs):
        self.gain = tf.Variable(initial_value=1.34, dtype=tf.complex128)
        self.poles = tf.Variable(initial_value=[
                     (-0.05429768147702485+1.4172655611120915e-05j),
                     (0.2917622403739163+0.7731812636894612j),
                     (0.8768382244780407-0.31120458350060115j),
                     (0.6598943546882394-0.46728573398560225j)
                 ], dtype=tf.complex128)
        self.zeros = tf.Variable(initial_value=[
                     (0.635496172349615+0.14499945287904842j),
                     (0.30987058966944614-0.8574194617385421j),
                     (0.5721096307971768-2.2915816453724273e-05j)
                 ], dtype=tf.complex128)
        cell = PZCell(self.gain, self.poles, self.zeros)
        super().__init__(cell, **kwargs)
        
class CARLayer(RNNLayer):
    def __init__(self, linear=False, erb_per_step=0.5, **kwargs):
        self.cc = CARCell(linear=linear, erb_per_step=erb_per_step)
        super().__init__(self.cc, **kwargs)
        
class SNRModel(tf.keras.Sequential):    
    def __init__(self, linear=False, **kwargs):
        super().__init__(**kwargs)
        self.ear_layer = EarLayer() 
        self.add(self.ear_layer)
        self.car_layer = CARLayer(linear=linear)
        self.add(self.car_layer)
        self.add(tf.keras.layers.Permute((2,1)))
        self.add(FFTLayer())
        self.add(SNRLayer())
    
class LoudnessPredictorModel(SNRModel):
    def __init__(self, linear=False, **kwargs):
        super().__init__(**kwargs)
        self.loudness_predictor_layer = LoudnessPredictorLayer()
        self.add(self.loudness_predictor_layer)
        
def assertequal(a1, a2):
    npa1 = np.array(a1)
    npa2 = np.array(a2)
    if npa1.shape != npa2.shape:
        raise ValueError(f'a1 has shape {npa1.shape} and a2 has shape {npa2.shape}')
    nonequal = (npa1 != npa2).nonzero()
    nonequal1 = npa1[nonequal]
    nonequal2 = npa2[nonequal]
    if len(nonequal1) > 0:
        raise ValueError(f'a1[{tuple(np.array(nonequal)[:,0])}] ({nonequal1[0]}) != a2{tuple(np.array(nonequal)[:,0])} ({nonequal2[0]})')

In [ ]:
window_size = 2048
bin_width = 48000 / window_size
print(bin_width)
mask_start = [80, 200]
probe = [60, 1000]
steps = 500
width = 20
examples = []
overlap = [None, None]
for idx, mask_center in enumerate(np.linspace(mask_start[1], probe[1] * 2, steps)):
    if mask_center + width + bin_width >= probe[1] - width - bin_width and overlap[0] is None:
        overlap[0] = idx
    if overlap[0] is not None and overlap[1] is None and mask_center - width - bin_width > probe[1] + width + bin_width:
        overlap[1] = idx
    examples.append(synthesize_example([probe[1] - width, probe[1] + width], probe[0], [[probe[0], probe[1] - width, probe[1] + width], [mask_start[0], mask_center - width, mask_center + width]], -70))
examples = prepare_examples(examples).batch(steps)
print('Overlap within', overlap)
for ex in examples.take(1):
    art_ex = ex

In [ ]:
lpm = LoudnessPredictorModel()
(worst_index, worst_error, worst_example) = plot_pred(lpm, examples)
print('worst error at', worst_index, worst_error)

In [ ]:
window_size = 2048
noise = make_noise(1, 980, 1020)

plt.figure(figsize=(10,4))
plt.plot(np.arange(len(noise)), noise)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,4))
plt.xlim((0,100))
plt.plot(np.arange(len(noise)//2), np.abs(np.fft.fft(noise))[:len(noise)//2])
plt.tight_layout()
plt.show()

window_size = 2048
ex = synthesize_example([980,1020], 40, [[-30,980, 1020], [-20, 540, 600]], -70, window_size=window_size)

plt.figure(figsize=(10,4))
plt.plot(np.arange(window_size), ex[:window_size,0])
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,4))
plt.plot(np.arange(window_size//2), np.abs(np.fft.fft(ex[:window_size, 0]))[:window_size//2])
plt.tight_layout()
plt.show()


In [ ]:
window_size = 2048
one_khz_examples = load_examples('/home/zond/DriveFileStream/My Drive/korvapuusti/listening_tests/modern_format/by_probe_center/1kHz/*/*.json').batch(512)
for ex in one_khz_examples.take(1):
    ex0 = ex
plt.figure(figsize=(10,4))
plt.plot(np.arange(window_size), np.abs(ex0[0][0,:window_size]))
plt.tight_layout()
plt.show()

In [ ]:
assertequal(list(itertools.combinations(np.arange(5),4)), [[0, 1, 2, 3], [0, 1, 2, 4], [0, 1, 3, 4], [0, 2, 3, 4], [1, 2, 3, 4]])

In [ ]:
pz_cell = PZCell(tf.constant(1.34, dtype=tf.complex128),
                 tf.constant([
                     (-0.05429768147702485+1.4172655611120915e-05j),
                     (0.2917622403739163+0.7731812636894612j),
                     (0.8768382244780407-0.31120458350060115j),
                     (0.6598943546882394-0.46728573398560225j)]),
                 tf.constant([
                     (0.635496172349615+0.14499945287904842j),
                     (0.30987058966944614-0.8574194617385421j),
                     (0.5721096307971768-2.2915816453724273e-05j)]))
inputs = tf.ones([2,1])
state = tuple(tf.zeros(shape=[inputs.shape[0], size]) for size in pz_cell.state_size)
inputs, state = pz_cell.call(inputs, state)
inputs, state = pz_cell.call(inputs, state)
inputs, state = pz_cell.call(inputs, state)
inputs, state = pz_cell.call(inputs, state)
inputs, state = pz_cell.call(inputs, state)
pz_cell.call(inputs, state)


In [ ]:
assertequal(coeffs(tf.constant([
    (-0.05429768147702485+1.4172655611120915e-05j),
    (0.2917622403739163+0.7731812636894612j),
    (0.8768382244780407-0.31120458350060115j),
    (0.6598943546882394-0.46728573398560225j),
    (-0.05429768147702485-1.4172655611120915e-05j),
    (0.2917622403739163-0.7731812636894612j),
    (0.8768382244780407+0.31120458350060115j),
    (0.6598943546882394+0.46728573398560225j)]), length=8), tf.constant([
    (1+0j), 
    (-3.548394276126343+5.551115123125783e-17j), 
    (5.916185211745642-3.885780586188048e-16j), 
    (-5.950823836910118-2.7755575615628914e-17j), 
    (3.819250686844303+3.3306690738754696e-16j),
    (-1.4222617112159746+8.326672684688674e-17j), 
    (0.1942351079538039-6.938893903907228e-18j), 
    (0.03639413827830768-2.8189256484623115e-18j), 
    (0.0011396243277688343+1.0842021724855044e-19j)]))

In [ ]:
ear_layer = EarLayer()
impulse = np.zeros([3, 10, 1], dtype=np.float64)
impulse[:,0,:] = 1
got = ear_layer(impulse)
want_per_batch = [[[0.0], [0.0], [1.34], [0.6880115972617817], [0.5898481920156424], [0.2653899964171016], [-0.1783844973373454], [-0.5351619597902066], [-0.6439493737887534], [-0.5374802978821727]]]
want = tf.constant(np.concatenate([want_per_batch, want_per_batch, want_per_batch], axis=0))
assertequal(got, want)

In [ ]:
plot_pz(EarLayer())

In [ ]:
ear_layer = EarLayer()
impulse = np.zeros([3, 2048, 1], dtype=np.float64)
impulse[:,0,:] = 1
t = time.time()
got = ear_layer(impulse)
print(time.time() - t)
t = time.time()
got = ear_layer(impulse)
print(time.time() - t)
plot_many_z(np.array([np.fft.fft(got[0,:,0])]))

In [ ]:
car_layer = CARLayer(linear=True)
impulse = np.zeros([3, 2048, 1], dtype=np.float64)
impulse[:,0,:] = 1
impulse = tf.constant(impulse, name='impulse')
t = time.time()
got = car_layer(impulse)
print(time.time() - t)
t = time.time()
got = car_layer(impulse)
print(time.time() - t)
got = tf.transpose(got, [0,2,1])
plot_many_z(np.fft.fft(got[0]))

In [ ]:
car_layer = CARLayer(erb_per_step=1)
impulse = np.zeros([3, 2048, 1], dtype=np.float64)
impulse[:,0,:] = 1
impulse = tf.constant(impulse, name='impulse')
got = car_layer(impulse)
got = tf.transpose(got, [0,2,1])
plot_many_z(np.fft.fft(got[0]))

In [ ]:
snrm = SNRModel(linear=True)
got = snrm(ex0[0])
plot_snr(got[0])

In [ ]:
snrm = SNRModel(linear=False)
got = snrm(ex0[0])
plot_snr(got[0])

In [ ]:
lpm = LoudnessPredictorModel()
(worst_index, worst_error, worst_example) = plot_pred(lpm, one_khz_examples)
print('worst error at', worst_index, worst_error)

In [ ]:
plt.figure(figsize=(10,5))
plt.tight_layout()
plt.xlim((0,150))
plt.ylim((-60,50))
plt.plot(np.arange(window_size//2), 20 * np.log10(1e-14 + np.abs(np.fft.fft(ex0[0][307][:,0]))[:window_size//2]))
plt.show()

In [ ]:
lpm = LoudnessPredictorModel()
impulse = np.zeros([3, 2048, 1], dtype=np.float64)
impulse[:,0,:] = 1
impulse = tf.constant(impulse, name='impulse')
lpm(impulse)
tape = tf.GradientTape()
with tape:
    res = lpm(impulse)
tape.gradient(res, [
    lpm.loudness_predictor_layer._loudness_scale,
    lpm.loudness_predictor_layer._loudness_offset,
    lpm.car_layer.cc._high_f_damping_compression,
    lpm.car_layer.cc._zero_ratio,
    lpm.car_layer.cc._min_zeta,
    lpm.car_layer.cc._erb_constant_0,
    lpm.car_layer.cc._velocity_scale,
    lpm.car_layer.cc._v_offset,
    lpm.ear_layer.gain,
    lpm.ear_layer.poles,
    lpm.ear_layer.zeros
])